In [1122]:
%pylab inline
import sys
sys.path.append("..")
from loaders import imagenet_load
from importlib import reload
print(imagenet_load)
from conv import multigpu
from loaders.imagenet_load import orient
import conv
from conv import filter_gen, opt, multigpu
import numpy as np
from utils import misc
from utils import linalg
from utils import exputil
from numba import jit
import math
import concurrent.futures as fs
from pylab import imshow
import copy
from skimage.filters import gabor_kernel
from skimage.transform import rescale

Populating the interactive namespace from numpy and matplotlib
<module 'loaders.imagenet_load' from '../loaders/imagenet_load.py'>


/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['linalg', 'copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
NUM_CLASSES = 10

In [3]:

def class_to_tarball_name_train(classname):
    classname = classname.strip()
    return "/home/ubuntu/imagenet-train/{0}-scaled.tar".format(classname)

def class_to_tarball_name_val(classname):
    classname = classname.strip()
    return "/home/ubuntu/imagenet-validation/val-{0}-scaled.tar".format(classname)



In [462]:
%time loader_train = imagenet_load.ImagenetLoader(NUM_CLASSES, classes_path="../loaders/classes", tarball_func=class_to_tarball_name_train, mmap_loc="/dev/shm/imagenet_train", n_procs=16)
%time loader_val = imagenet_load.ImagenetLoader(NUM_CLASSES, classes_path="../loaders/classes", tarball_func=class_to_tarball_name_val, mmap_loc="/dev/shm/imagenet_train", n_procs=16)

%time X_train = loader_train.load_all().astype('float64')
%time X_test = loader_val.load_all().astype('float64')

%time X_train = normalize_images(X_train)
%time X_test = normalize_images(X_test)

%time mean_image = np.mean(X_train, axis=0)
%time X_train -= mean_image
%time X_test -= mean_image

There exists a matrix at /dev/shm/imagenet_train


CPU times: user 20 ms, sys: 2.23 s, total: 2.25 s
Wall time: 2.55 s


There exists a matrix at /dev/shm/imagenet_train


CPU times: user 24 ms, sys: 1.94 s, total: 1.96 s
Wall time: 2.16 s
Loaded 0 classes into mem
CPU times: user 3.28 s, sys: 6.61 s, total: 9.89 s
Wall time: 12.9 s
Loaded 0 classes into mem
CPU times: user 148 ms, sys: 2.17 s, total: 2.32 s
Wall time: 2.63 s
PRE PROCESSING
CPU times: user 15.7 s, sys: 13.7 s, total: 29.3 s
Wall time: 29.3 s
PRE PROCESSING
CPU times: user 604 ms, sys: 528 ms, total: 1.13 s
Wall time: 1.12 s
CPU times: user 4.14 s, sys: 0 ns, total: 4.14 s
Wall time: 4.13 s
CPU times: user 3.67 s, sys: 4 ms, total: 3.67 s
Wall time: 3.67 s
CPU times: user 116 ms, sys: 0 ns, total: 116 ms
Wall time: 119 ms


In [463]:
y_train = loader_train.Y
y_test = loader_val.Y

In [464]:
num_pixels = X_train.shape[1]*X_train.shape[2]*X_train.shape[3]

In [921]:
num_pixels

196608

In [470]:
%time X_train_flat = X_train.reshape(X_train.shape[0], -1) / np.sqrt(num_pixels)
%time X_test_flat = X_test.reshape(X_test.shape[0], -1)  / np.sqrt(num_pixels)

In [922]:
X_train_flat

array([[ -2.03486261e-04,  -2.20109386e-04,  -1.79659784e-04, ...,
          2.96890380e-04,   3.02852941e-04,   3.03848768e-04],
       [ -5.44885060e-05,  -7.43944000e-05,  -8.84689705e-05, ...,
         -4.26344990e-04,  -3.78707015e-04,  -3.23238227e-04],
       [ -1.14394453e-04,  -1.16142693e-04,  -1.16599023e-04, ...,
          7.24585938e-04,   7.26829778e-04,   7.27825605e-04],
       ..., 
       [  1.54596231e-04,   2.10270672e-04,   8.34844417e-05, ...,
         -3.36185703e-04,  -2.88003717e-04,  -3.21461499e-04],
       [  3.04049863e-04,   2.72234284e-04,   2.86811623e-04, ...,
         -6.31025631e-05,  -9.59372845e-05,  -1.45053689e-04],
       [ -2.93694146e-04,  -5.33300616e-05,   2.58411080e-04, ...,
         -7.02395444e-05,  -1.18966720e-04,  -4.78857462e-05]])

In [472]:
K_train_flat = X_train_flat.dot(X_train_flat.T)
K_test_flat = X_test_flat.dot(X_train_flat.T)

In [475]:
np.mean(X_train_flat, axis=1)

array([ -6.21157495e-21,  -6.77626358e-21,  -9.03501810e-21, ...,
         2.37169225e-20,  -5.19513541e-20,  -9.03501810e-21])

In [473]:
results_flat = {}

In [476]:
for reg in [1e-4, 1e-2, 1e-1, 1e1, 10, 1000, 10000]:
    acc = opt.trainAndEvaluateDualModel(KTrain=K_train_flat, KTest=K_test_flat, labelsTrain=y_train.astype('int'), 
                                                                      labelsTest=y_test.astype('int'),
                                                                      reg=reg)
    print(reg, acc)
    results_flat[(0,reg)] = acc

Learning Dual Model
reg is 0.0001
0.0001 (0.99761538461538457, 0.23000000000000001)
Learning Dual Model
reg is 0.01
0.01 (0.8223076923076923, 0.34599999999999997)
Learning Dual Model
reg is 0.1
0.1 (0.52746153846153843, 0.376)
Learning Dual Model
reg is 10.0
10.0 (0.34069230769230768, 0.32200000000000001)
Learning Dual Model
reg is 10
10 (0.34069230769230768, 0.32200000000000001)
Learning Dual Model
reg is 1000
1000 (0.29215384615384615, 0.27000000000000002)
Learning Dual Model
reg is 10000
10000 (0.29069230769230769, 0.27200000000000002)


In [479]:
for gamma in [1e3]:
    K_train_rbf_pixels = linalg.computeRBFGramMatrix(X_train_flat, X_train_flat, gamma=gamma)
    K_test_rbf_pixels = linalg.computeRBFGramMatrix(X_test_flat, X_train_flat, gamma=gamma)
    for reg in [1e-5, 1e-4, 1e-1, 1, 10, 100]:
        acc = opt.trainAndEvaluateDualModel(KTrain=K_train_rbf_pixels, KTest=K_test_rbf_pixels, labelsTrain=y_train.astype('int'), 
                                                                      labelsTest=y_test.astype('int'),
                                                                      reg=reg)
        print(gamma, reg, acc)
        results_flat[(gamma,reg)] = acc

Learning Dual Model
reg is 1e-05
1000.0 1e-05 (0.99815384615384617, 0.38)
Learning Dual Model
reg is 0.0001
1000.0 0.0001 (0.99815384615384617, 0.38)
Learning Dual Model
reg is 0.1
1000.0 0.1 (0.99807692307692308, 0.378)
Learning Dual Model
reg is 1
1000.0 1 (0.998, 0.38)
Learning Dual Model
reg is 10
1000.0 10 (0.98969230769230765, 0.38)
Learning Dual Model
reg is 100
1000.0 100 (0.97769230769230764, 0.35399999999999998)


In [459]:
def normalize_images(images, min_divisor=1e-8):
    orig_shape = images.shape
    images = images.reshape(images.shape[0], -1)
    print("PRE PROCESSING")
    n_images = images.shape[0]
    # Zero mean every feature
    images = images - np.mean(images, axis=1)[:,np.newaxis]
    # Normalize
    image_norms = np.linalg.norm(images, axis=1)/55.0
    # Get rid of really small norms
    image_norms[np.where(image_norms < min_divisor)] = 1
    # Make features unit norm
    images_normalized = images/image_norms[:,np.newaxis]
    return images_normalized.reshape(orig_shape)

In [800]:
def normalize_patches(patches, min_divisor=1e-8, zca_bias=0.1):

    orig_shape = patches.shape
    patches = patches.reshape(patches.shape[0], -1)
    print("PRE PROCESSING")
    n_patches = patches.shape[0]

    # Zero mean every feature
    patches = patches - np.mean(patches, axis=1)[:,np.newaxis]

    # Normalize
    patch_norms = np.linalg.norm(patches, axis=1)/55.0

    # Get rid of really small norms
    patch_norms[np.where(patch_norms < min_divisor)] = 1

    # Make features unit norm
    patches = patches/patch_norms[:,np.newaxis]


    whitening_means = np.mean(patches, axis=0)
    
    data_means = np.mean(patches, axis=1)


    zeroCenterPatches = (patches - whitening_means[np.newaxis, :])

    patchesCovMat = 1.0/n_patches * zeroCenterPatches.T.dot(zeroCenterPatches)

    (E,V) = np.linalg.eig(patchesCovMat)

    E += zca_bias
    sqrt_zca_eigs = np.sqrt(E)
    
    inv_sqrt_zca_eigs = np.diag(np.power(sqrt_zca_eigs, -1))
    
    global_ZCA = V.dot(inv_sqrt_zca_eigs).dot(V.T)
    
    patches_normalized = (patches - whitening_means).dot(global_ZCA)

    return patches_normalized.reshape(orig_shape)




@jit(nopython=True, nogil=True, cache=True)
def __grab_patches(images, patch_size=6, tot_patches=1e6, seed=0, scale_pixels=1.0/255.0):
    np.random.seed(seed)
    tot_patches = int(tot_patches)
    im_idxs = np.random.choice(images.shape[0], tot_patches)
    idxs_x = np.random.choice(images.shape[3] - patch_size - 1, tot_patches)
    idxs_y = np.random.choice(images.shape[2] - patch_size - 1, tot_patches)
    idxs_x += int(math.ceil(patch_size/2))
    idxs_y += int(math.ceil(patch_size/2))
    patches = np.zeros((tot_patches, images.shape[1], patch_size, patch_size), dtype=np.float32)
    for i, (im_idx, idx_x, idx_y) in enumerate(zip(im_idxs, idxs_x, idxs_y)):
        out_patch = patches[i, :, :, :]
        grab_patch_from_idx(images[im_idx], idx_x, idx_y, patch_size, out_patch, scale_pixels)
    return patches
    
@jit(nopython=True, nogil=True)
def grab_patch_from_idx(im, idx_x, idx_y, patch_size, outpatch, scale_pixels=1.0/255.0):
    sidx_x = int(idx_x - patch_size/2)
    eidx_x = int(idx_x + patch_size/2)
    sidx_y = int(idx_y - patch_size/2)
    eidx_y = int(idx_y + patch_size/2)
    outpatch[:,:,:] = im[:, sidx_x:eidx_x, sidx_y:eidx_y,].astype(np.float32)*scale_pixels
    return outpatch

def grab_patches(images, patch_size=6, tot_patches=1e6, seed=0, max_threads=50, scale_pixels=1.0/255.0):
    idxs = misc.chunk_idxs(images.shape[0], max_threads)
    tot_patches = int(tot_patches)
    patches_per_thread = int(tot_patches/max_threads)
    with fs.ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = []
        for i,(sidx, eidx) in enumerate(idxs):
            futures.append(executor.submit(__grab_patches, images[sidx:eidx, :], 
                                           patch_size=patch_size,
                                           tot_patches=patches_per_thread,
                                           seed=seed+i))
            
        results = np.vstack(list(map(lambda x: x.result(), futures)))
    return results 

In [897]:
patch_size = 12
%time patches = grab_patches(X_train, patch_size=patch_size)

CPU times: user 28.3 s, sys: 1.49 s, total: 29.8 s
Wall time: 1.5 s


In [898]:
patches_normalized = normalize_patches(patches)

PRE PROCESSING


In [899]:
reload(filter_gen)
patches_mmap_data = np.memmap("/dev/shm/imagenet_patches", shape=patches.shape, dtype=patches.dtype, mode="w+")
np.copyto(patches_mmap_data, patches)
patches_mmap = multigpu.MmapArray(patches_mmap_data, mode="r+")

patches_normalized_mmap_data = np.memmap("/dev/shm/imagenet_patches_normalized", shape=patches.shape, dtype=patches.dtype, mode="w+")
np.copyto(patches_normalized_mmap_data, patches_normalized)
patches_normalized_mmap = multigpu.MmapArray(patches_normalized_mmap_data, mode="r+")

fg_patches = filter_gen.make_empirical_filter_gen(patches_mmap, seed=0)
fg_patches_normalized = filter_gen.make_empirical_filter_gen(patches_normalized_mmap, seed=0)

In [1162]:
def make_gabor_filter_gen(freq=0.2, patch_size=12, channels=3, seed=0):
    np.random.seed(seed)
    def gabor_filter_gen(num_filters):
        out = np.zeros((num_filters, channels, patch_size, patch_size))
        for f in range(num_filters):
            for c in range(channels):
                theta = np.random.uniform(0, 2*math.pi)
                gamma = np.random.uniform(0.1, 1)
                gk = gabor_kernel(freq, theta, gamma).real
                scale = float(patch_size)/gk.shape[0]
                gk_scaled = rescale(gk, scale)
                out[f,c]  = gk_scaled
        return out
    return gabor_filter_gen



In [1145]:
gabor_filters = make_gabor_filter_gen()(1024)

In [1188]:
gabor_filters

array([[[[ -8.61619518e-06,  -6.32587313e-05,   3.24320346e-04,
            1.06092527e-04,  -1.09860440e-04,   2.16927646e-06],
         [  7.18469215e-05,  -8.00250617e-04,  -7.22630607e-04,
            2.82063256e-03,  -4.36004437e-05,  -1.03271744e-04],
         [  2.63063036e-04,   8.00012203e-04,  -8.21327293e-03,
           -8.18871016e-05,   2.53643826e-03,  -1.38814377e-04],
         [ -1.38814377e-04,   2.53643826e-03,  -8.18871016e-05,
           -8.21327293e-03,   8.00012203e-04,   2.63063036e-04],
         [ -1.03271744e-04,  -4.36004437e-05,   2.82063256e-03,
           -7.22630607e-04,  -8.00250617e-04,   7.18469215e-05],
         [  2.16927646e-06,  -1.09860440e-04,   1.06092527e-04,
            3.24320346e-04,  -6.32587313e-05,  -8.61619518e-06]],

        [[  3.14266727e-05,   3.15391422e-05,  -4.54185110e-04,
            4.21528570e-04,  -2.51998067e-06,  -3.49871074e-05],
         [ -1.61864662e-04,  -1.20366025e-04,   2.23071067e-03,
           -2.15160966e-03,   5

In [1156]:
gpu_handler = multigpu.MultiGpuHandler(16)
gpu_handler.kill_all_gpu_processes()
gpu_handler.start_and_wait_for_gpu_init()

In [1171]:
conv_args_template = \
{ 
  "num_feature_batches":1,
  "data_batch_size":8,
  "feature_batch_size":1024,
  "pool_size":70,
  "pool_type":"avg",
  "pool_stride":45,
  "patch_size":patch_size,
  "pad":0,
  "bias":0,
  "conv_stride":2,
  "scale_pixels": 1.0/255.0
}
X_train_normalized = np.memmap("/dev/shm/X_train_normalized", dtype='float32', shape=X_train.shape, mode="w+")
X_test_normalized = np.memmap("/dev/shm/X_test_normalized", dtype='float32', shape=X_test.shape, mode="w+")
np.copyto(X_train_normalized, X_train.astype('float32'))
np.copyto(X_test_normalized, X_test.astype('float32'))

X_train_mmap = multigpu.MmapArray(X_train_normalized, mode="r+")
X_test_mmap = multigpu.MmapArray(X_test_normalized, mode="r+")

mmap_out_train_shape = conv.conv_compute_output_shape(data=X_train_mmap, **conv_args_template)
X_train_lift = np.memmap("/dev/shm/X_train_lift", mode="w+", dtype="float32", shape=mmap_out_train_shape)

mmap_out_test_shape = conv.conv_compute_output_shape(data=X_test_mmap, **conv_args_template)
X_test_lift = np.memmap("/dev/shm/X_test_lift", mode="w+", dtype="float32", shape=mmap_out_test_shape)

In [1172]:
np.mean(np.var(patches_normalized, axis=(1,2,3)))

0.70112854

In [1173]:
fg_patches = filter_gen.make_empirical_filter_gen(patches_normalized_mmap, MIN_VAR_TOL=0.1, seed=0)

In [1174]:
np.mean(np.var(fg_patches(1024), axis=(1,2,3)))

1.6505256

In [1175]:
X_train_lift_mmap = multigpu.MmapArray(X_train_lift, mode="r+")
X_test_lift_mmap = multigpu.MmapArray(X_test_lift, mode="r+")


In [1176]:
train_chunk_idxs = misc.chunk_idxs(X_train.shape[0], 16)
test_chunk_idxs = misc.chunk_idxs(X_test.shape[0], 16)

In [1177]:
gpu_results = []
for gpu, (sidx, eidx) in list(zip(gpu_handler.gpus, train_chunk_idxs)):
        fg_gabor =  make_gabor_filter_gen()
        conv_args = conv_args_template.copy()
        conv_args["filter_gen"] = fg_gabor
        X_train_mmap_gpu = copy.copy(X_train_mmap)
        X_train_mmap_gpu.idxs = (sidx, eidx)
        X_train_out_gpu = copy.copy(X_train_lift_mmap)
        X_train_out_gpu.idxs = (sidx, eidx)
        conv_args['data'] = X_train_mmap_gpu
        conv_args['output'] = X_train_out_gpu
        gpu_result = gpu.submit_async(conv.conv_mmap, **conv_args)
        gpu_results.append(gpu_result)

for gpu_result in gpu_results:
    %time gpu_result.result()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 21.1 s
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 401 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 333 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 113 ms
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 347 µs
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 312 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 324 µs
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 322 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 222 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 225 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 90.1 ms
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 448 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 294 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 335 µs
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 310 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time

In [1178]:
gpu_results = []
for gpu, (sidx, eidx) in list(zip(gpu_handler.gpus, test_chunk_idxs)):
        fg_gabor =  make_gabor_filter_gen()
        conv_args = conv_args_template.copy()
        conv_args["filter_gen"] = fg_gabor
        X_test_mmap_gpu = copy.copy(X_test_mmap)
        X_test_mmap_gpu.idxs = (sidx, eidx)
        X_test_out_gpu = copy.copy(X_test_lift_mmap)
        X_test_out_gpu.idxs = (sidx, eidx)
        conv_args['data'] = X_test_mmap_gpu
        conv_args['output'] = X_test_out_gpu
        gpu_result = gpu.submit_async(conv.conv_mmap, **conv_args)
        gpu_results.append(gpu_result)

for gpu_result in gpu_results:
    %time gpu_result.result()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.52 s
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 606 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 552 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 432 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 420 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 389 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 515 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 408 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 102 ms
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 499 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 419 µs
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 411 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 444 µs
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 537 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 362 µs
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time:

In [1179]:
X_train_lift = X_train_lift_mmap.load()
X_test_lift = X_test_lift_mmap.load()

In [1180]:
np.mean(np.sum(X_train_lift == 0, axis=(1,2,3)))/(len(X_train_lift[0].ravel()))

0.0016672467614850428

In [1182]:
X_train_lift

memmap([[[[  1.68576310e-06,   1.76462834e-06,   2.15534010e-06],
         [  8.42288046e-07,   1.10896849e-06,   1.83296640e-06],
         [  1.36745120e-06,   1.18212404e-06,   1.58550938e-06]],

        [[  1.36457527e-06,   1.05265053e-06,   1.08104234e-06],
         [  1.22339350e-06,   1.28804015e-06,   1.43423972e-06],
         [  1.17704315e-06,   1.49169273e-06,   1.33975198e-06]],

        [[  8.10402616e-06,   4.88999058e-06,   4.96770463e-06],
         [  3.30113221e-06,   2.65290032e-06,   4.43238650e-06],
         [  2.56084127e-06,   2.98058285e-06,   4.43225326e-06]],

        ..., 
        [[  1.12608689e-06,   7.60248952e-07,   9.12704309e-07],
         [  5.59873513e-07,   4.79085827e-07,   5.07451091e-07],
         [  4.33952636e-07,   4.81558743e-07,   3.75638592e-07]],

        [[  4.66529571e-07,   6.07429456e-07,   6.75793501e-07],
         [  8.79428342e-07,   1.01144224e-06,   7.87393844e-07],
         [  7.90464753e-07,   8.69270593e-07,   3.62175541e-07]],



In [1183]:
num_feats = conv_args_template['feature_batch_size']*conv_args_template['num_feature_batches']

In [1184]:
%time X_train_lift_flat = X_train_lift.reshape(X_train_lift.shape[0], -1).astype('float64') / np.sqrt(num_feats)

CPU times: user 632 ms, sys: 908 ms, total: 1.54 s
Wall time: 1.54 s


In [1185]:
%time X_test_lift_flat = X_test_lift.reshape(X_test_lift.shape[0], -1).astype('float64') / np.sqrt(num_feats)

CPU times: user 40 ms, sys: 20 ms, total: 60 ms
Wall time: 61.8 ms


In [1186]:
%time K_train_conv = X_train_lift_flat.dot(X_train_lift_flat.T) 
%time K_test_conv = X_test_lift_flat.dot(X_train_lift_flat.T)

CPU times: user 1min 53s, sys: 55.4 s, total: 2min 49s
Wall time: 6.01 s
CPU times: user 9.3 s, sys: 3.61 s, total: 12.9 s
Wall time: 423 ms


In [1189]:
X_test_lift

memmap([[[[  1.58109140e-06,   1.45059391e-06,   1.81394523e-06],
         [  1.07277515e-06,   9.33922877e-07,   1.33725371e-06],
         [  8.09303970e-07,   1.05893491e-06,   1.23484165e-06]],

        [[  5.19300283e-06,   4.69473162e-06,   5.96304153e-06],
         [  4.03942477e-06,   3.48478284e-06,   4.39495670e-06],
         [  2.42085662e-06,   3.14486670e-06,   3.74502383e-06]],

        [[  5.81503218e-06,   6.15005274e-06,   7.77288369e-06],
         [  4.43296540e-06,   4.56608586e-06,   5.86736041e-06],
         [  2.61551872e-06,   3.38705149e-06,   3.89128900e-06]],

        ..., 
        [[  3.75242166e-06,   3.68401197e-06,   4.54085512e-06],
         [  3.05350295e-06,   3.04033665e-06,   3.39478834e-06],
         [  1.87235798e-06,   2.33977471e-06,   2.40090276e-06]],

        [[  2.56324256e-06,   2.53998155e-06,   3.11629810e-06],
         [  1.86308182e-06,   1.81827863e-06,   2.24448786e-06],
         [  1.47691674e-06,   1.73546937e-06,   1.74710976e-06]],



In [1187]:
for reg in [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-1, 1, 1e1, 1e5]:
    acc = opt.trainAndEvaluateDualModel(KTrain=K_train_conv, KTest=K_test_conv, labelsTrain=y_train.astype('int'), 
                                                                      labelsTest=y_test.astype('int'), reg=reg)
    print(reg, acc)

Learning Dual Model
reg is 1e-08
1e-08 (0.37892307692307692, 0.108)
Learning Dual Model
reg is 1e-07
1e-07 (0.28415384615384615, 0.10000000000000001)
Learning Dual Model
reg is 1e-06
1e-06 (0.14238461538461539, 0.10000000000000001)
Learning Dual Model
reg is 1e-05


KeyboardInterrupt: 

In [1028]:
results_conv = {}
for gamma in [5e-2,1e-1, 1, 10, 100]:
    K_train_rbf_conv = linalg.computeRBFGramMatrix(X_train_lift, X_train_lift, gamma=gamma)
    K_test_rbf_conv = linalg.computeRBFGramMatrix(X_test_lift, X_train_lift, gamma=gamma)
    for reg in [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]:
        acc = opt.trainAndEvaluateDualModel(KTrain=K_train_rbf_conv, KTest=K_test_rbf_conv, labelsTrain=y_train.astype('int'), 
                                                                      labelsTest=y_test.astype('int'),
                                                                      reg=reg)
        print(gamma, reg, acc)
        results[(gamma,reg)] = acc

Learning Dual Model
reg is 1e-05
0.05 1e-05 (0.99769230769230766, 0.69799999999999995)
Learning Dual Model
reg is 0.0001
0.05 0.0001 (0.98038461538461541, 0.70199999999999996)
Learning Dual Model
reg is 0.001
0.05 0.001 (0.85853846153846158, 0.70599999999999996)
Learning Dual Model
reg is 0.01
0.05 0.01 (0.72246153846153849, 0.68799999999999994)
Learning Dual Model
reg is 0.1
0.05 0.1 (0.6154615384615385, 0.59799999999999998)
Learning Dual Model
reg is 1e-05
0.1 1e-05 (0.99792307692307691, 0.68200000000000005)
Learning Dual Model
reg is 0.0001
0.1 0.0001 (0.99407692307692308, 0.70599999999999996)
Learning Dual Model
reg is 0.001
0.1 0.001 (0.92723076923076919, 0.70199999999999996)
Learning Dual Model
reg is 0.01
0.1 0.01 (0.77246153846153842, 0.68999999999999995)
Learning Dual Model
reg is 0.1
0.1 0.1 (0.65407692307692311, 0.61799999999999999)
Learning Dual Model
reg is 1e-05
1 1e-05 (0.998, 0.60599999999999998)
Learning Dual Model
reg is 0.0001
1 0.0001 (0.99807692307692308, 0.6139999

KeyboardInterrupt: 

In [ ]:
X_test_lift

In [34]:
K_test_rbf_pixels

array([[ 1.        ,  0.99926612,  0.99909939, ...,  0.99929579,
         0.99898554,  0.99931603],
       [ 0.99926612,  1.        ,  0.99913969, ...,  0.99927931,
         0.9989939 ,  0.99928697],
       [ 0.99909939,  0.99913969,  1.        , ...,  0.99925441,
         0.99911216,  0.99934219],
       ..., 
       [ 0.99905437,  0.99906212,  0.99914661, ...,  0.9995055 ,
         0.99951988,  0.99959997],
       [ 0.99926577,  0.99925152,  0.99916929, ...,  0.99948263,
         0.99917748,  0.9994573 ],
       [ 0.99912478,  0.99928218,  0.99900119, ...,  0.99943302,
         0.99906306,  0.99931051]])

In [30]:
X_train_flat.shape

(13000, 196608)

In [999]:
exputil.snapshot()

/bin/bash -c 'cd /home/ubuntu ; tar -cf ./picture_web_2017-04-11-07:56:40.625613.tar pictureweb --exclude-from <(find pictureweb -size +3M)'


In [892]:
im = imread("/home/ubuntu/tiny-imagenet-200/train/n01443537/images/n01443537_0.JPEG")

In [895]:
im.shape

(64, 64, 3)

In [923]:
fc7 = np.load("/home/ubuntu/imagenet-fc7/imagenet_features_alexnet_fc7.npz")

In [924]:
X_train_fc7 = fc7["X_train"]
y_train_fc7 = fc7["y_train"]

In [925]:
X_test_fc7 = np.load("/home/ubuntu/imagenet-fc7/imagenet_test_features_alexnet_fc7.npy")
y_test_fc7 = np.load("/home/ubuntu/imagenet-fc7/imagenet_test_labels.npy")

In [967]:
idxs = np.where(y_train_fc7 < 10)[0]
idxs_test = np.where(y_test_fc7 < 10)[0]
X_train_fc7_small = X_train_fc7[idxs, :]
y_train_fc7_small = y_train_fc7[idxs].astype('int')
X_test_fc7_small = X_test_fc7[idxs_test, :]
y_test_fc7_small = y_test_fc7[idxs_test].astype('int')


In [961]:
opt.trainAndEvaluatePrimalModel(X_train_fc7_small, X_test_fc7_small, y_train_fc7_small, y_test_fc7_small, reg=1000000)

X SHAPE  (12622, 4096)
Computing XTX
Done Computing XTX


(0.95642528917762637, 0.91400000000000003)

In [952]:
labels = y_train_fc7_small

In [953]:
y = np.eye(max(labels) + 1)[labels]

In [965]:
XtX = X_train_fc7_small.T.dot(X_train_fc7_small)

In [979]:
misc.rreload(softmax)
misc.rreload(opt)
from opt import softmax
softmax.softmax_gn(X_train_fc7_small, y_train_fc7_small, X_test_fc7_small, y_test_fc7_small, XtX, step_size=1)

step 1 took 0.15542125701904297
Compting train_preds 2
Iter: 0, Train Accuracy: 0.9988115987957534, Test Accuracy: 0.9
step 1 took 0.1811075210571289
Compting train_preds 2
Iter: 1, Train Accuracy: 0.9988115987957534, Test Accuracy: 0.9
step 1 took 0.17215442657470703
Compting train_preds 2
Iter: 2, Train Accuracy: 0.9988115987957534, Test Accuracy: 0.902
step 1 took 0.16565442085266113
Compting train_preds 2
Iter: 3, Train Accuracy: 0.9988908255427033, Test Accuracy: 0.9
step 1 took 0.1684567928314209
Compting train_preds 2
Iter: 4, Train Accuracy: 0.998970052289653, Test Accuracy: 0.9
step 1 took 0.17835235595703125
Compting train_preds 2
Iter: 5, Train Accuracy: 0.9992077325305023, Test Accuracy: 0.902
step 1 took 0.16594862937927246
Compting train_preds 2
Iter: 6, Train Accuracy: 0.9993661860244019, Test Accuracy: 0.902
step 1 took 0.16495776176452637
Compting train_preds 2
Iter: 7, Train Accuracy: 0.9994454127713516, Test Accuracy: 0.902
step 1 took 0.1683351993560791
Compting tra

KeyboardInterrupt: 

In [972]:
opt.softmax

AttributeError: module 'conv.opt' has no attribute 'softmax'